In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import os

import pandas as pd
import numpy as np
import pydicom

In [2]:
image_dir = R"C:\Users\u1126257\Desktop\aiml_training\capstone\rsna-pneumonia-detection-challenge\stage_2_train_images"
labels_dir = R"C:\Users\u1126257\Desktop\aiml_training\capstone\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"

In [3]:

df = pd.read_csv(labels_dir)
X = df['patientId']
y = df['Target']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1)

In [5]:
X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [6]:
X_train[0]

'020380f8-5c5a-4ded-bdf3-9ce3036945b4'

In [7]:
def load_img_into_ndarray(pid):
    file_name = pid + ".dcm"
    dcm_path = os.path.join(image_dir, file_name)
    # dcm_path = image_dir + "\\" + pid + ".dcm"
    data_row_img_data = pydicom.read_file(dcm_path)
    ar = data_row_img_data.pixel_array
    # ar = np.reshape(ar,(1024,1024,1))
    # ar = ar.reshape(1024,1024,1)
    ar = ar[..., np.newaxis] 
    # ar = ar.reshape(ar.shape + (1,))
    
    # ar = np.expand_dims(ar, axis=0)
    # print(ar.shape)
    
    return ar/255.0

p = '00322d4d-1c29-4943-afc9-b6754be640eb'
# load_img_into_ndarray(p)/255.0

In [8]:
l = []
for i in range(len(X_train)):
    # X_train[i] = load_img_into_ndarray(X_train[i])
    l.append(load_img_into_ndarray(X_train[i]))

X_train = np.array(l)

In [9]:
l = []
for i in range(len(X_test)):
    # X_test[i] = load_img_into_ndarray(X_test[i])
    l.append(load_img_into_ndarray(X_test[i]))

X_test = np.array(l)

In [11]:
# X_train, X_test = X_train / 255.0, X_test / 255.0

In [12]:
# class_names = ['Normal', 'Lung Opacity']
# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(X_train[i])
#     # The CIFAR labels happen to be arrays, 
#     # which is why you need the extra index
#     plt.xlabel(class_names[y_train[i]])
# plt.show()

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (7, 7), activation='relu', input_shape=(1024, 1024,1), ))
model.add(layers.MaxPooling2D((5, 5)))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation = 'sigmoid'))
# model.add(layers.Dense(num of classes,activation = 'softmax'))

In [15]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1018, 1018, 32)    1600      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 203, 203, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 199, 199, 64)      51264     
                                                                 
 dense (Dense)               (None, 199, 199, 16)      1040      
                                                                 
 dense_1 (Dense)             (None, 199, 199, 4)       68        
                                                                 
 flatten (Flatten)           (None, 158404)            0         
                                                        

In [17]:
# For multiple category
# Dense(n,activation='softmax')
# model.compile(loss='categorical_crossentropy') / sparse_categorical_crossentropy

# For 2 category
# Dense(1,activation='sigmoid')
# model.compile(loss='binary_crossentropy')


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=4, 
                    validation_data=(X_test, y_test))

Epoch 1/4
18/18 [==============================] - 457s 24s/step - loss: 0.8326 - accuracy: 0.5536 - val_loss: 0.6849 - val_accuracy: 0.5750
Epoch 2/4
18/18 [==============================] - 405s 22s/step - loss: 0.6838 - accuracy: 0.5607 - val_loss: 0.6326 - val_accuracy: 0.7292
Epoch 3/4
18/18 [==============================] - 401s 22s/step - loss: 0.5587 - accuracy: 0.7321 - val_loss: 0.5874 - val_accuracy: 0.7417
Epoch 4/4
18/18 [==============================] - 401s 22s/step - loss: 0.4773 - accuracy: 0.7857 - val_loss: 0.5619 - val_accuracy: 0.7875


In [18]:
model_name = "not_78"
model.save('saved_model\my_model' + model_name)
model.save_weights('checkpoint\my_checkpoint' + model_name)

INFO:tensorflow:Assets written to: saved_model\my_model2_78acc\assets


INFO:tensorflow:Assets written to: saved_model\my_model2_78acc\assets


In [83]:
df1 = pd.read_csv(labels_dir)[100:150]
X1 = df1['patientId']
y1 = df1['Target']

In [84]:
X1, y1 = np.array(X1), np.array(y1)

In [85]:
l = []
for i in range(len(X1)):
    # X_train[i] = load_img_into_ndarray(X_train[i])
    l.append(load_img_into_ndarray(X1[i]))

X1 = np.array(l)

In [86]:
X1.shape

(100, 1024, 1024, 1)

In [87]:
model.evaluate(X1,  y1, verbose=2)

4/4 - 13s - loss: 3.7669 - accuracy: 0.8400 - 13s/epoch - 3s/step


[3.7668936252593994, 0.8399999737739563]

In [88]:
def do_pred(x):
    # v = load_img_into_ndarray(x)
    l = []
    v = x
    l.append(v)
    l = np.array(l)
    # l = l / 255.0
    return l

In [90]:
valll = X1[0]
l = []
l.append(valll)
l = np.array(l)
l.shape

(1, 1024, 1024, 1)

In [96]:
model.predict(l)[0][0] > 0.5

1/1 [==============================] - 0s 154ms/step


False

In [92]:
do_pred(X1[i]).shape

(1, 1024, 1024, 1)

In [105]:
def pred_fun(file_name):
    l = []
    l.append(load_img_into_ndarray(file_name))
    l = np.array(l)
    # model = getmodel()
    return model.predict(l)[0][0]


pred_fun(p)

1/1 [==============================] - 0s 158ms/step


0.0

In [93]:
for i in range(len(X1)):
    print(model.predict(do_pred(X1[i])))
    print(y1[i])

1/1 [==============================] - 0s 300ms/step
[[0.43160477]]
0
1/1 [==============================] - 0s 199ms/step
[[0.478861]]
0
1/1 [==============================] - 0s 217ms/step
[[0.3781229]]
0
1/1 [==============================] - 0s 199ms/step
[[0.28921664]]
0
1/1 [==============================] - 0s 204ms/step
[[0.54275185]]
1
1/1 [==============================] - 0s 192ms/step
[[0.54275185]]
1
1/1 [==============================] - 0s 202ms/step
[[0.43974972]]
0
1/1 [==============================] - 0s 168ms/step
[[0.34419945]]
0
1/1 [==============================] - 0s 169ms/step
[[0.5015181]]
1
1/1 [==============================] - 0s 231ms/step
[[0.5015181]]
1
1/1 [==============================] - 0s 217ms/step
[[0.4388795]]
0
1/1 [==============================] - 0s 191ms/step
[[0.36623394]]
0
1/1 [==============================] - 0s 203ms/step
[[0.37281543]]
0
1/1 [==============================] - 0s 185ms/step
[[0.41740704]]
0
1/1 [=====================